# GOMS 데이터 회귀 분석 - 직업군 예측

In [53]:
# !pip install openpyxl
# !pip install xlrd


In [54]:
import pandas as pd
import numpy as np
import openpyxl
import xlrd
import pickle
import joblib
import matplotlib.pyplot as plt
import seaborn as sns


In [55]:
data = pd.read_excel('./goms_filtered_re.xlsx', engine='openpyxl') # 각자 경로로 바꾸기
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18163 entries, 0 to 18162
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   schoolType       18163 non-null  int64  
 1   schoolType_str   18163 non-null  object 
 2   major            18163 non-null  object 
 3   graduY           18163 non-null  int64  
 4   age              18163 non-null  float64
 5   internExp        12692 non-null  float64
 6   toeicScore       5406 non-null   float64
 7   tospGrade        1187 non-null   float64
 8   trainingHr       3981 non-null   float64
 9   jobseekYN        18163 non-null  int64  
 10  certNum          10350 non-null  float64
 11  mainJobCate      13295 non-null  float64
 12  mainJobCate_str  13295 non-null  object 
 13  midJobCate       13295 non-null  float64
 14  midJobCate_str   13295 non-null  object 
 15  subJobCate       13295 non-null  float64
 16  subJobCate_str   13295 non-null  object 
dtypes: float64(9

In [56]:
# null 확인
# data.isnull().sum()

In [57]:
# 결측치 0으로 대체
data = data.fillna(0)

## 종속변수가 main

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [59]:
data_main = data.drop(['schoolType_str','subJobCate', 'mainJobCate_str', 'midJobCate', 'midJobCate_str', 'subJobCate_str'],axis=1)
data_main

,schoolType,major,graduY,age,internExp,toeicScore,tospGrade,trainingHr,jobseekYN,certNum,mainJobCate
0,1,비즈니스일본어과,2018,23.500000,0.0,0.0,0.0,0.0,1,0.0,6.0
1,1,일본어과,2018,21.750000,0.0,0.0,0.0,240.0,1,0.0,0.0
2,1,일본어과,2018,21.916667,0.0,800.0,0.0,0.0,0,1.0,3.0
3,1,관광일어과,2018,25.583333,0.0,0.0,0.0,0.0,0,0.0,6.0
4,1,관광일어과,2018,21.750000,0.0,0.0,0.0,0.0,1,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
18158,2,뮤지컬전공,2017,27.333333,0.0,0.0,0.0,0.0,1,0.0,0.0
18159,2,교회음악학과,2018,25.083333,0.0,0.0,0.0,0.0,0,0.0,3.0
18160,2,교회음악과,2018,26.750000,0.0,0.0,0.0,0.0,0,0.0,7.0
18161,1,유아교육학과,2018,28.583333,0.0,0.0,0.0,0.0,0,0.0,0.0


In [60]:
X_ohe1 = pd.get_dummies(data_main, columns=['major'])
y_main = X_ohe1['mainJobCate']
x_main = X_ohe1.drop(['mainJobCate'],axis=1)

x_main


,schoolType,graduY,age,internExp,toeicScore,tospGrade,trainingHr,jobseekYN,certNum,major_3D입체영상과,...,major_회계정보과(3년과정),major_회계정보학과,major_회계학과,major_회계학전공,major_회화과,major_회화전공,major_회화학과,major_회화학과(서양화전공),major_휴먼복지학부,major_힐빙복지계열
0,1,2018,23.500000,0.0,0.0,0.0,0.0,1,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2018,21.750000,0.0,0.0,0.0,240.0,1,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2018,21.916667,0.0,800.0,0.0,0.0,0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2018,25.583333,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2018,21.750000,0.0,0.0,0.0,0.0,1,1.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18158,2,2017,27.333333,0.0,0.0,0.0,0.0,1,0.0,0,...,0,0,0,0,0,0,0,0,0,0
18159,2,2018,25.083333,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
18160,2,2018,26.750000,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
18161,1,2018,28.583333,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_main, y_main, test_size=0.3, random_state=0)
main_rfr = RandomForestRegressor(max_depth = 3,random_state=12)
main_rfr.fit(X_train, y_train)


In [ ]:
y_train_pred = main_rfr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)
y_test_pred = main_rfr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)

In [38]:
joblib.dump(main_rfr,'./main_randomforest.pkl')

['./main_randomforest.pkl']

## 종속변수가 mid

In [39]:
data_mid = data.drop(['schoolType_str','subJobCate', 'mainJobCate_str', 'mainJobCate', 'midJobCate_str', 'subJobCate_str'],axis=1)

In [40]:
X_ohe2 = pd.get_dummies(data_mid, columns=['major'])
y_mid = X_ohe2['midJobCate']
x_mid = X_ohe2.drop(['midJobCate'],axis=1)



Train MSE: 538.1987
Test MSE: 552.0407


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_mid, y_mid, test_size=0.3, random_state=0)
mid_rfr = RandomForestRegressor(max_depth = 3,random_state=12)
mid_rfr.fit(X_train, y_train)


In [ ]:
y_train_pred = mid_rfr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)
y_test_pred = mid_rfr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)

In [41]:
joblib.dump(mid_rfr,'./mid_randomforest.pkl')

['./mid_randomforest.pkl']

In [49]:
load_mainRdf = joblib.load('./main_randomforest.pkl')
load_midRdf = joblib.load('./mid_randomforest.pkl')

print(load_mainRdf.score(X_test, y_test), load_midRdf.score(X_test, y_test))

-0.48870081970329005 0.0440329165248996


In [52]:
y_hat = load_mainRdf.predict(X)

plt.figure(figsize=(10, 5))
ax1 = sns.kdeplot(y, label="y")
ax2 = sns.kdeplot(y_hat, label="y_hat", ax=ax1)
plt.legend()
plt.show()

NameError: name 'X' is not defined